### this is my master notebook for processing BCH data
- last edited July 15, 2020

In [1]:
#run this cell when online
!pip install xlrd

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## install dependencies and set data paths

In [3]:
import pandas as pd
import numpy as np

import os
import re
from collections import Counter

In [4]:
#use this cell when working online
path = '/floyd/home/ed-triage'
data_path = '/floyd/home/data'

In [5]:
#use this cell when working from home
path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/ed-triage'
data_path = '/Users/jjaskolkambp/Desktop/machine learning/my_projects/data/ED triage project/bch'

In [6]:
#loading in the entire spreadsheet as a dataframe
clin = pd.read_excel(data_path + '/BCH ED visits FY1819_with additions.xlsx')

In [7]:
clin.shape

(136993, 27)

In [8]:
clin.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading'],
      dtype='object')

In [9]:
clin.dropna(subset = ['SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory', 'Triage Date & Time', 'Reg Date & Time'], inplace = True)
clin.shape

(102492, 27)

## The next few cells are for getting a feel of the data in the different columns and how they might be useful

In [9]:
discharge_count = Counter(clin['DischargeDisposition']); discharge_count

Counter({17: 88232,
         62: 263,
         63: 1419,
         7: 9283,
         6: 532,
         30: 217,
         40: 242,
         16: 499,
         13: 1199,
         64: 354,
         90: 23,
         12: 65,
         8: 139,
         14: 19,
         61: 49,
         9: 1,
         72: 7})

In [10]:
discharge_count2 = Counter(clin['DischargeDispositionDesc']); discharge_count2

Counter({'Discharge to private home, condo, apt without support service/referral': 88232,
         'Left at his/her own risk post-initial treatment': 263,
         'Left After Triage': 1419,
         'Admit to reporting facility as inpatient to another unit from amb care': 9283,
         'Admit to reporting facility as inpatient to SCU or OR from amb care': 532,
         'Transfer to Residential care': 217,
         'Transfer to Group/supportive living': 242,
         'Discharge to private home, condo, apt with support service/referral': 499,
         'Intra-facility transfer to ED': 1199,
         'Left After Initial Assessment': 354,
         'Transfer to correctional facility': 23,
         'Intra-facility transfer to day surgery': 65,
         'Transfer to another acute care facility directly from amb care': 139,
         'Intra-facility transfer to clinic': 19,
         'Left at his/her own risk following registration': 49,
         'Transfer to another non-acute care facility dir

In [11]:
comp_count = Counter(clin['PatientService']); comp_count

Counter({nan: 92747,
         'Orthopaedics': 138,
         'MH Child & Adolescent': 289,
         'INTENSIVE CARE UNIT': 170,
         'General Medicine': 3566,
         'Respirology': 410,
         'Genitourinary': 505,
         'General Surgery': 1069,
         'Cardiology': 1408,
         'Mental Health': 418,
         'Paediatrics': 823,
         'Palliative': 74,
         'Oncology': 322,
         'Neurology': 269,
         'ALC General Medicine': 96,
         'ALC Neurology': 45,
         'ALC Respirology': 13,
         'Acute Care for Elders': 29,
         'ALC Cardiology': 40,
         'Neonatal Retro Transfers': 4,
         'ALC Orthopaedics': 46,
         'AD PALLIATIVE': 14,
         'ALC General Surgery': 12,
         'Obstetrics': 19,
         'ALC Oncology': 4,
         'AD Respirology': 1,
         'AD Oncology': 5,
         'ALC ACUTE CARE FOR EDLERS': 2,
         'AD Cardiology': 1,
         'ALC Palliative': 3,
         'AD General Surgery': 1})

## refactoring some of the cells so they are clean and useful

### tabular data items

In [51]:
o2sat = [item if item[-1] != '%' else item[:-1] for item in clin['O2Saturation_LastEDReading'].astype('str')]
o2sat = [np.nan if item == 'nan' else float(item) for item in o2sat] 
clin['o2sat'] = o2sat

In [52]:
pulse = clin['Pulse_LastEDReading']
pulse = [re.sub("[^0-9]", "", str(item)) for item in pulse]
pulse = [np.nan if item == '' else float(item) for item in pulse] 
#set(pulse)
clin['pulse'] = pulse

In [53]:
temp = clin['Temperature_LastEDReading']
temp = [re.sub("[^0-9]", "", str(item)) for item in temp]
temp = [np.nan if item == '' else float(item) for item in temp] 
temp = [item/10 if item > 100 else item for item in temp]
temp = [np.nan if item > 40  else item for item in temp]
#set(temp)
clin['temp'] = temp

In [54]:
def bptrans(bp):
    if pd.isnull(bp) or len(bp.split('/')) !=2:
        return [np.nan, np.nan]
    res = []
    for x in bp.split('/'):
        try:
            float(x)
            res =  [float(x) for x in bp.split('/')]
        except: 
            res =  [np.nan, np.nan]
        return res


In [55]:
clin['BP'] = clin.BloodPressure_LastEDReading.map(bptrans)

clin[['systolic', 'diastolic']] = pd.DataFrame(clin.BP.tolist(), index= clin.index)
clin["Gender"] = clin.GenderDesc.map(lambda x: 1 if x == "Male" else 0)

### clean subjective notes

In [10]:
#this is to clean the text in the subjective notes column (a little bit)
def fixencode(s):
    if pd.isnull(s):
        s = "None"
    s = s[14:]
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s = s.replace('<LF><LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = re.sub(r'\bOR\b', 'operation', s)
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('h/o', 'history of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    
    s = re.sub(r'\bhrs\b', 'hours', s)
    s = re.sub(r'\bwks\b', 'weeks', s)
    s = re.sub(r'\byrs\b', 'years', s)
    
    s = re.sub(r'\bhx\b', 'history', s)
    s = re.sub(r'\bdx\b', 'diagnosis', s)
    
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('lmp', 'last menstrual period')
    s = re.sub(r'\biud\b', 'intrauterine contraceptive device', s)
    s = re.sub(r'\biucd\b', 'intrauterine contraceptive device', s)
    s = re.sub(r'\bvb\b', 'vaginal bleeding', s)
    s = s.replace('w/', 'with')
    s = s.replace('@', ' at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\br\s*\/\s*o','rule out',s)
    s = re.sub(r'\bc\s*\/\s*o','complains of',s)
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\blt\b','left ',s)
    s = re.sub(r'\brt\b','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = re.sub(r'\bhtn\b', 'hyptertension', s)
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub('(?<=[a-zA-Z])-(?=[a-zA-Z])', ' ', s)  #I think this is mean to manage dashes, but I have done this a couple of other ways
    s = re.sub(r'\bwic\b', 'walk in clinic ',s)
    s = re.sub(r'\bcabg\b', 'coronary artery bypass ',s)
    s = re.sub(r'\bgerd\b', 'gastroesophageal reflux ',s)
    s = re.sub(r'\badhd\b', 'attention deficit disorder ',s)
    s = re.sub(r'\badd\b', 'attention deficit disorder ',s)
    s = re.sub(r'\bcopd\b', 'chronic obstructive pulmonary disease ',s)
    s = re.sub(r'\bems\b', 'emergency medical services ',s)
    s = re.sub(r'\bmvc\b', 'motor vehicle collision  ',s)
    s = re.sub(r'\bgp\b', 'family doctor  ',s)
    s = re.sub(r'\bloc\b', 'loss of consciousness ',s)
    s = re.sub(r'\bsts\b', 'states  ',s)
    s = re.sub(r'\bmvc', 'motor vehicle collision ',s)
    s = re.sub(r'\bmva', 'motor vehicle collision ',s)
    s = re.sub(r'\bnsr\b', 'normal sinus rhythm ',s)
    s = re.sub(r'\bbp\b', 'blood pressure ',s)
    s = re.sub(r'\betoh\b', 'alcohol ',s)
    s = re.sub(r'\bhtn\b', 'hypertension ',s)
    s = re.sub(r'\btia\b', 'transient ischemic attack ',s)
    s = re.sub(r'\biddm\b', 'type one diabetes ',s)
    s = re.sub(r'\bniddm\b', 'type two diabetes ',s)
    s = re.sub(r'\biddm\b', 'type one diabetes ',s)
    s = re.sub(r'\bdmi\b', 'type one diabetes ',s)
    s = re.sub(r'\bdm i\b', 'type one diabetes ',s)
    s = re.sub(r'\bdm1\b', 'type one diabetes ',s)
    s = re.sub(r'\bniddm\b', 'type two diabetes ',s)
    s = re.sub(r'\bdmii\b', 'type two diabetes ',s)
    s = re.sub(r'\bdm2\b', 'type two diabetes ',s)
    s = re.sub(r'\bdm ii\b', 'type two diabetes ',s)
    s = re.sub(r'\bcrf\b', 'chronic renal failure ',s)
    s = re.sub(r'\bckd\b', 'chronic renal failure ',s)
    s = re.sub(r'\bchf\b', 'congestive heart failure',s)
    s = re.sub(r'\bcad\b', 'coronary artery disease ',s)
    s = re.sub(r'\bmi\b', 'myocardial infarction  ',s)
    s = re.sub(r'\bsbo\b', 'small bowel obstruction ',s)
    s = re.sub(r'\blbo\b', 'large bowel obstruction ',s)
    s = re.sub(r'\baaa\b', 'abdominal aortic aneurysm',s)
    s = re.sub(r'\bocd\b', 'obsessive compulsive disorder',s)
    s = re.sub(r'\bpvd\b', 'peripheral vascular disease',s)
    s = re.sub(r'\bdvt\b', 'deep vein thrombosis',s)
    s = re.sub(r'\bibs\b', 'irritable bowel syndrome',s)
    s = re.sub(r'\basd\b', 'atrial septal defect',s)
    s = re.sub(r'\bvsd\b', 'ventricular septal defect',s)
    s = re.sub(r'\bpci\b', 'percutaneous coronary intervention',s)
    s = re.sub(r'\brsv\b', 'respiratory syincitial virus',s)
    s = re.sub(r'\btbi\b', 'traumatic brain injury',s)
    s = re.sub(r'\btb\b', 'tuberculosis',s)
    s = re.sub(r'\bacs\b', 'acute coronary syndrome',s)
    s = re.sub(r'\boa\b', 'osteoarthritis',s)
    s = re.sub(r'\bdnr\b', 'do not resuscitate',s)
    s = re.sub(r'\bpid\b', 'pelvic inflammatory disease',s)
    s = re.sub(r'\bpe\b', 'pulmonary embolus',s)
    s = re.sub(r'\babd\b', 'abdomen',s)
    s = re.sub(r'\bcva\b', 'stroke',s) 
    s = re.sub(r'\bca\b', 'cancer ',s)
    s = re.sub(r'and/or', 'or', s)
    s = re.sub(r'\bneg', 'negative', s)
    s = re.sub(r's/b', 'sent by', s)
    s = re.sub(r'nausea\s*\/\s*vomiting', 'nausea or vomiting', s)
    s = re.sub(r';',',',s)
    s = re.sub(r'\baaox3\b', 'alert and oriented', s)
    s = re.sub(r'\bcsm\b', ' circulation, sensation and movement ',s)
    s = re.sub(r'\bsz\b', ' seizure ',s)
    s = re.sub(r'\bhgb\b', 'hemoglobin', s)
    s = re.sub(r'\basa\b', 'aspirin', s)
    s = re.sub(r'\butd\b', 'up to date', s)
    s = re.sub(r'\brx\b', 'treatment', s)
    s = re.sub(r'\bnpo\b', 'nothing by mouth', s)
    s = re.sub(r'\bd/c\b', 'discharge', s)
    s = re.sub(r'\bo2\b', 'oxygen', s)
    s = re.sub(r'\biv\b', 'intravenous', s)
    s = re.sub(r'\brxn\b', 'reaction', s)
    s = re.sub(r'\bh/a\b', 'headache', s)
    
    
    #managing punctuation
    s = s.replace("--", "-")
    s = s.replace('-', ' ')
    s = s.replace('=', ' ')
    #s = re.sub(' of ', ' ', s)
    #s = re.sub(' to ', ' ', s)
    #s = re.sub(' in ', ' ', s)
    #s = re.sub(' at ', ' ', s)
    s = re.sub('\)', '', s)
    s = re.sub(r'\s+,',',', s)
    s = re.sub(r'\s+\.','.', s)

    #puncts = '!"$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    punctuation_spacer = str.maketrans({key: f"{key} " for key in ".,"})
    s = s.translate(punctuation_spacer)
    
    puncts = '!"$&\'()*+-:,.;<=>?@[\\]^_`{|}~'  
    s = s.translate(str.maketrans(' ',' ',puncts))
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    return s

In [11]:
clin['CleanSubjectiveNotes'] = clin.SubjectiveNotes.map(fixencode)

In [12]:
subj_notes = list(clin['CleanSubjectiveNotes'])

In [13]:
for note in subj_notes[5000:5020]:
    print()
    print(note)
    print()


patient has had a fib for over 18 years rate used be high frequent visits to emergency room has tapered off in last few years recently has had more palpitations and worse with exercise and feels heart racing for over 1 week thought it would resolve does biking and and exercise daily no meds currently has not seen cardiologist in last few years patient was on flecamide by mouth has not seen cardio in over 10 years denies cp


as per patient was playing lacrosse this am had a collision patient now in emergency room complains of right rib pain pain worse when taking deep breath and when lifting right arm up 


patient in emergency room complains of 6/10 constant epigastric pain since last night patient states pain started after eating veal and steamed rice last night patient also complains of stiffness/tightness to calves and forearms patient complains of nausea denies vomiting patient refuses antiemetics in triage lbm this am very little and hard patient complains of increasing epigastr

### cleaning text of medical hx (for NLP)

In [14]:
#this is to clean the text in the medical history column (a little bit)
def fixmedhx(s):
    s = s[16:]
    s =s.replace('<LT><LF>',' ')
    s = s.replace('<LT>LF>',' ')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics ')
    s = s.replace(' htn ', ' hypertension ')
    s = re.sub(r'\btyl\b', 'tylenol ', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant ',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant ',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub(r'#\s', 'fracture ', s)
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [15]:
clin['CleanMedicalHistory'] = clin.MedicalHistory.map(fixmedhx)

### clean and split medical history into list of comorbidites

In [16]:
history = list(clin['MedicalHistory'].str[16:].astype('str'))

In [17]:
#this is to clean the text in the medical history column (a little bit)
def fixhx(s):
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s = s.replace('<LF><LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = re.sub(r'\bOR\b', 'operation', s)
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('h/o', 'history of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    
    s = re.sub(r'\bhrs\b', 'hours', s)
    s = re.sub(r'\bwks\b', 'weeks', s)
    s = re.sub(r'\byrs\b', 'years', s)
    
    s = re.sub(r'\bhx\b', 'history', s)
    s = re.sub(r'\bdx\b', 'diagnosis', s)
    
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('lmp', 'last menstrual period')
    s = re.sub(r'\biud\b', 'intrauterine contraceptive device', s)
    s = re.sub(r'\biucd\b', 'intrauterine contraceptive device', s)
    s = re.sub(r'\bvb\b', 'vaginal bleed', s)
    s = s.replace('w/', 'with')
    s = s.replace('@', ' at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\br\s*\/\s*o','rule out',s)
    s = re.sub(r'\bc\s*\/\s*o','complains of',s)
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\blt\b','left ',s)
    s = re.sub(r'\brt\b','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = re.sub(r'\bhtn\b', 'hyptertension', s)
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub('(?<=[a-zA-Z])-(?=[a-zA-Z])', ' ', s)  #I think this is mean to manage dashes, but I have done this a couple of other ways
    s = re.sub(r'\bwic\b', 'walk in clinic ',s)
    s = re.sub(r'\bcabg\b', 'coronary artery bypass ',s)
    s = re.sub(r'\bgerd\b', 'gastroesophageal reflux ',s)
    s = re.sub(r'\badhd\b', 'attention deficit disorder ',s)
    s = re.sub(r'\badd\b', 'attention deficit disorder ',s)
    s = re.sub(r'\bcopd\b', 'chronic obstructive pulmonary disease ',s)
    s = re.sub(r'\bems\b', 'emergency medical services ',s)
    s = re.sub(r'\bmvc\b', 'motor vehicle collision  ',s)
    s = re.sub(r'\bgp\b', 'family doctor  ',s)
    s = re.sub(r'\bloc\b', 'loss of consciousness ',s)
    s = re.sub(r'\bsts\b', 'states  ',s)
    s = re.sub(r'\bmvc', 'motor vehicle collision ',s)
    s = re.sub(r'\bmva', 'motor vehicle collision ',s)
    s = re.sub(r'\bnsr\b', 'normal sinus rhythm ',s)
    s = re.sub(r'\bbp\b', 'blood pressure ',s)
    s = re.sub(r'\betoh\b', 'alcohol ',s)
    s = re.sub(r'\bhtn\b', 'hypertension ',s)
    s = re.sub(r'\btia\b', 'transient ischemic attack ',s)
    s = re.sub(r'\biddm\b', 'type one diabetes ',s)
    s = re.sub(r'\bniddm\b', 'type two diabetes ',s)
    s = re.sub(r'\biddm\b', 'type one diabetes ',s)
    s = re.sub(r'\bdmi\b', 'type one diabetes ',s)
    s = re.sub(r'\bdm i\b', 'type one diabetes ',s)
    s = re.sub(r'\bdm1\b', 'type one diabetes ',s)
    s = re.sub(r'\bniddm\b', 'type two diabetes ',s)
    s = re.sub(r'\bdmii\b', 'type two diabetes ',s)
    s = re.sub(r'\bdm2\b', 'type two diabetes ',s)
    s = re.sub(r'\bdm ii\b', 'type two diabetes ',s)
    s = re.sub(r'\bcrf\b', 'chronic renal failure ',s)
    s = re.sub(r'\bckd\b', 'chronic renal failure ',s)
    s = re.sub(r'\bchf\b', 'congestive heart failure',s)
    s = re.sub(r'\bcad\b', 'coronary artery disease ',s)
    s = re.sub(r'\bmi\b', 'myocardial infarction  ',s)
    s = re.sub(r'\bsbo\b', 'small bowel obstruction ',s)
    s = re.sub(r'\blbo\b', 'large bowel obstruction ',s)
    s = re.sub(r'\baaa\b', 'abdominal aortic aneurysm',s)
    s = re.sub(r'\bocd\b', 'obsessive compulsive disorder',s)
    s = re.sub(r'\bpvd\b', 'peripheral vascular disease',s)
    s = re.sub(r'\bdvt\b', 'deep vein thrombosis',s)
    s = re.sub(r'\bibs\b', 'irritable bowel syndrome',s)
    s = re.sub(r'\basd\b', 'atrial septal defect',s)
    s = re.sub(r'\bvsd\b', 'ventricular septal defect',s)
    s = re.sub(r'\bpci\b', 'percutaneous coronary intervention',s)
    s = re.sub(r'\brsv\b', 'respiratory syncytial virus',s)
    s = re.sub(r'\btbi\b', 'traumatic brain injury',s)
    s = re.sub(r'\btb\b', 'tuberculosis',s)
    s = re.sub(r'\bacs\b', 'acute coronary syndrome',s)
    s = re.sub(r'\boa\b', 'osteoarthritis',s)
    s = re.sub(r'\bdnr\b', 'do not resuscitate',s)
    s = re.sub(r'\bpid\b', 'pelvic inflammatory disease',s)
    s = re.sub(r'\bpe\b', 'pulmonary embolus',s)
    s = re.sub(r'\babd\b', 'abdomen',s)
    s = re.sub(r'\bcva\b', 'stroke',s) 
    s = re.sub(r'\bca\b', 'cancer ',s)
    s = re.sub(r'and/or', 'or', s)
    s = re.sub(r'\bneg', 'negative', s)
    s = re.sub(r's/b', 'sent by', s)
    s = re.sub(r'nausea\s*\/\s*vomiting', 'nausea or vomiting', s)
    s = re.sub(r';',',',s)
    s = re.sub(r'aaox3', 'alert and oriented', s)
    s = re.sub(r'csm', ' circulation, sensation and movement ',s)
    s = re.sub(r'\bsz\b', ' seizure ',s)
    s = re.sub(r'hgb', 'hemoglobin', s)
    s = re.sub(r'\basa\b', 'aspirin', s)
    s = re.sub(r'\butd\b', 'up to date', s)
    s = re.sub(r'\brx\b', 'treatment', s)
    s = re.sub(r'\bnpo\b', 'nothing by mouth', s)
    s = re.sub(r'd/c', 'discharge', s)
    s = re.sub(r'o2', 'oxygen', s)
    s = re.sub(r'\biv\b', 'intravenous', s)
    s = re.sub(r'rxn', 'reaction', s)
    
    
    #managing punctuation
    s = s.replace("--", "-")
    s = s.replace('-', ' ')
    s = re.sub(' of ', ' ', s)
    s = re.sub(' to ', ' ', s)
    s = re.sub(' in ', ' ', s)
    s = re.sub(' at ', ' ', s)
    s = re.sub('\)', '', s)
    s = re.sub(r'\s+,',',', s)
    s = re.sub(r'\s+\.','.', s)

    puncts = '!"$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    s = s.translate(str.maketrans('','',puncts))
        
    #punctuation_spacer = str.maketrans({key: f"{key} " for key in ".,"})
    #s = s.translate(punctuation_spacer)
    
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    return s

In [18]:
replacements = (',','<LT>LF>', '.', ';', '/')
drop_list = ('"', '?', 'had', 'has', 'have', 'left', 'right', 'sided','weeks', 'history of', 'h/o','and', 'acute')   #added all but first 2 options may 30, 2020
split_hist = []
for item in history:
    for r in replacements:
        item = item.replace(r, '::')
        
    item = item.split('::')
    item = [re.sub(r"(\b[a-z]\b)\s([a-z])", r"\1-\2", s) for s in item]  #to handle a fib, c section, h pylori
    item = [re.sub(r'\(.*?\)\ *', '', s) for s in item]  #remove punctuation
    item = [re.sub(r'\b\d+\b', '', s) for s in item]  #remove numbers
    item = [re.sub(r'%', '', s) for s in item]  #remove percent sign
    item = [re.sub(r'\'', '',s) for s in item]  #remove '
    item = [re.sub(r'\b[0-9]\w+''', '', s) for s in item]  #remove words that start with numbers
    item = [re.sub(r'^[^\w#]*', '',s) for s in item]    #remove leading symbols except fracture
    item = [re.sub(r'^-\s*', '',s) for s in item] #remove leading dashes
    item = [s.strip('-') for s in item] #remove leading and trailing dashes

    
    item = [fixhx(s) for s in item]   #run the cleaning function above to get rid of some of the abbreviation
    
    
    for d in drop_list:
        item = [x.replace(d,'') for x in item]   #get rid of the words from the blacklist above
    
    item = [x.strip().lower() for x in item if x != 'nan']   #get rid of white spaces
    item = [x for x in item if len(x) > 2]
    item = list(filter(None, item))
    
    split_hist.append(item)
    
    
 #split hist is a list of lists, where each entry is the list of past medical history conditions for each patient

In [19]:
#this is to clean the text in the medical history column (a little bit)
def fixhx_old(s):
    s =s.replace('<LT><LF>','')
    s = s.replace('<LT>LF>','')
    s =s.replace('Pt.', 'patient')
    s =s.replace('Pt', 'patient')
    s = s.lower()
    s = s.replace('pt.', 'patient')
    s = s.replace('y/o', 'year old')
    s = s.replace('c/o', 'complains of')
    s = s.replace('sob', 'shortness of breath')
    s = s.replace('c/p', 'chest pain')
    s = s.replace('hrs', 'hours')
    s = s.replace('hx', 'history')
    s = s.replace('n/v', 'nausea and/or vomiting')
    s = s.replace('a/e', 'air entry')
    s = s.replace('a/o', 'alert and oriented')
    s = s.replace('a&o', 'alert and oriented')
    s = s.replace('d/c', 'discharge')
    s = s.replace('u/s', 'ultrasound')
    s = s.replace('yrs', 'years')
    s = s.replace('lmp', 'last menstrual period')
    s = s.replace('w/', 'with')
    s = s.replace('@', 'at ')
    s = s.replace('b/c', 'because')
    s = re.sub(r'\bpt.','patient ',s)
    s = re.sub(r'\blt.','left ',s)
    s = re.sub(r'\brt.','right ',s)
    s = re.sub(r'\bmd.','doctor ',s)
    s = re.sub(r'\bra\b','room air ',s)
    s = re.sub(r'\bbp.','blood pressure ',s)
    s = re.sub(r'\bcp.','chest pain ',s)
    s = re.sub(r'\bfd.','family doctor ',s)
    s = s.replace('abx', 'antibiotics')
    s = s.replace(' htn ', ' hypertension ')
    s = re.sub(r'\btyl\b', 'tylenol', s)
    s = re.sub(r'\bprn\b', 'as needed ',s)
    s = re.sub(r'\bvag\b', 'vaginal ',s)
    s = re.sub(r'\bpo\b', 'by mouth ',s)
    s = re.sub(r'\bllq\b', 'left lower quadrant',s)
    s = re.sub(r'\brlq\b', 'right lower quadrant  ',s)
    s = re.sub(r'\bluq\b', 'left upper quadrant',s)
    s = re.sub(r'\bruq\b', 'right upper quadrant  ',s)
    s = re.sub(r'\buti\b', 'urinary tract infection ',s)
    s = re.sub(r'\brd', 'right digit number',s)
    s = re.sub(r'\bld', 'left digit number',s)
    s = re.sub(r'\br\b', 'right ',s)
    s = re.sub(r'\bl\b', 'left ',s)
    s = re.sub(r'\bed\b', 'emergency department ',s)
    s = re.sub(r'\ber\b', 'emergency room ',s)
    s = re.sub(r'\bbm\b', 'bowel movement ',s)
    s = re.sub(r'#\s', 'fracture ', s)
    
    #remove extra white spaces
    s = re.sub(r'\s+', ' ', s)

    #note that what's missing here is removal of punctuation
    return s

In [20]:
"""replacements = (',','<LT>LF>', '.')
drop_list = ('"', '?')
split_hist = []
for item in history:
    for r in replacements:
        item = item.replace(r, '::')
    
    item = item.split('::')
    
    item = [re.sub(r'\(.*?\)\ *', '', s) for s in item]
    
    item = [fixhx(s) for s in item]
    for d in drop_list:
        item = [x.replace(d,'') for x in item]
    
    item = [x.strip().lower() for x in item if x != 'nan']
    item = [x for x in item if len(x) > 2]
    
    item = list(filter(None, item))
    
    split_hist.append(item)"""
    

'replacements = (\',\',\'<LT>LF>\', \'.\')\ndrop_list = (\'"\', \'?\')\nsplit_hist = []\nfor item in history:\n    for r in replacements:\n        item = item.replace(r, \'::\')\n    \n    item = item.split(\'::\')\n    \n    item = [re.sub(r\'\\(.*?\\)\\ *\', \'\', s) for s in item]\n    \n    item = [fixhx(s) for s in item]\n    for d in drop_list:\n        item = [x.replace(d,\'\') for x in item]\n    \n    item = [x.strip().lower() for x in item if x != \'nan\']\n    item = [x for x in item if len(x) > 2]\n    \n    item = list(filter(None, item))\n    \n    split_hist.append(item)'

In [21]:
#my hideous way of limiting each diagnosis to 3 words long
split_hist2 = []
for item in split_hist:
    item2 = []
    for dx in item:
        if dx != 'no significant medical history':
            dx = re.sub(r"(\b[b-z]\b)\s([a-z])", r"\1-\2", dx)
            new_dx = ' '.join(dx.split()[:3])
            item2.append(new_dx)
        else:
            item2.append(dx)
    split_hist2.append(item2)

In [61]:
#rejoining them as a string with diagnoses separated by commas
join_hist = [', '.join(item) for item in split_hist2]

In [62]:
comorbids = [len(item) if item != ['no significant medical history'] else 0 for item in split_hist]

In [63]:
#replacing empty string with NaN
join_hist = [np.nan if item == 'nan' else item for item in join_hist]

In [64]:
len(join_hist)

102492

In [65]:
clin['pmhx'] = join_hist
clin['num_comorbids'] = comorbids

In [66]:
diagnoses = Counter()
for hx in split_hist2:   #note that this is done with my split modified split hist
    for item in hx:
        diagnoses[item] += 1

#gonna adapt the vocabulary to contain bi grams and trigrams
vocab = Counter()
for hx in split_hist2:   #note that this is done with my split modified split hist
    for item in hx:
        vocab[item] += 1
        word_list = item.split()
        if len(word_list) > 1:
            for word in word_list:
                vocab[word] += 1
                if len(word_list) == 3:
                    vocab[' '.join(word_list[:2])] +=1
                    vocab[' '.join(word_list[1:])] +=1
        


In [67]:
len(diagnoses), len(vocab)

(13737, 24945)

### exploring different diagnoses
- in here I initially made some code to make a data frame up of the top 6 diagnoses for each patient

In [28]:
diagnoses = Counter()
for hx in split_hist2:
    for item in hx:
        diagnoses[item] += 1

In [29]:
#there are 20464 different medical histories used in the dataset!!!
len(diagnoses)

13737

In [30]:
#sorted diagnoses
diagnoses.most_common(40)

[('no significant medical history', 44084),
 ('hypertension', 14904),
 ('high cholesterol', 13371),
 ('type two diabetes', 7190),
 ('high blood pressure', 4876),
 ('depression', 4589),
 ('asthma', 4317),
 ('anxiety', 3520),
 ('thyroid', 3269),
 ('diabetes', 2867),
 ('hypothyroid', 2480),
 ('type one diabetes', 2454),
 ('gastroesophageal reflux', 2211),
 ('hyptertension', 2134),
 ('acid reflux', 1719),
 ('arthritis', 1460),
 ('myocardial infarction', 1376),
 ('anemia', 1086),
 ('chronic obstructive pulmonary', 1044),
 ('stroke', 1033),
 ('schizophrenia', 839),
 ('kidney stones', 796),
 ('congestive heart failure', 744),
 ('seizure', 705),
 ('immunizations up date', 690),
 ('atrial fibrillation', 688),
 ('enlarged prostate', 679),
 ('attention deficit disorder', 656),
 ('gout', 655),
 ('pacemaker', 595),
 ('migraines', 586),
 ('dementia', 544),
 ('substance misuse', 536),
 ('breast cancer', 528),
 ('high chol', 499),
 ('bipolar', 479),
 ('osteoporosis', 466),
 ('chronic back pain', 466),

In [31]:
#this is a dictionary mapping the rank of the diagnosis to its name
dx_rank_dict = {item[0]:i+1 for i,item in enumerate(diagnoses.most_common())}

#this is a dictionary mapping the frequency of occurence to its name
dx_freq_dict = {item[0]:item[1]/len(diagnoses) for item in diagnoses.most_common()}

In [32]:
#trying to get a feel for how many diagoses are provided for each patient
split_hist_len = [len(item) for item in split_hist]
hxdf = pd.DataFrame(split_hist_len)

In [33]:
hxdf.describe()

,0
count,102492.000000
mean,1.786617
std,1.430899
min,0.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,27.000000


In [34]:
lengths = Counter(split_hist_len); lengths

Counter({1: 66376,
         5: 2870,
         6: 1455,
         2: 15179,
         3: 9430,
         9: 213,
         8: 378,
         4: 5483,
         7: 729,
         15: 10,
         0: 115,
         11: 68,
         10: 112,
         13: 18,
         12: 26,
         17: 7,
         14: 15,
         18: 1,
         16: 6,
         27: 1})

In [35]:
#most patients have only a single diagnosis, 80% have 2 or less, 96% have 6 or less
for k,v in sorted(lengths.items()):
    print (k, '{0:2f}%'.format(v*100/len(split_hist_len)))

0 0.112204%
1 64.762128%
2 14.809936%
3 9.200718%
4 5.349686%
5 2.800219%
6 1.419623%
7 0.711275%
8 0.368809%
9 0.207821%
10 0.109277%
11 0.066347%
12 0.025368%
13 0.017562%
14 0.014635%
15 0.009757%
16 0.005854%
17 0.006830%
18 0.000976%
27 0.000976%


In [89]:
#this will create a list for each item where the top 6 diagnoses are listed in order
ordered_hist = []
for item in split_hist:
    #print (item)
    #print (len(item))
    top_6 = []
    for hx in item:
        #print (hx)
        #print (dx_rank_dict[hx])
        #print (dx_rank_dict[hx], hx)
        if item == 'nan':
            top_6.append(np.nan)
        else:
            top_6.append((dx_rank_dict[hx], hx))
    #print (top_6)
    sort = sorted(top_6)
    #print (sort)
    sort2 = [ii for i,ii in sort]
    #print(sort2)
    if len(sort2) > 6:
        sort2 = sort2[:5]
    if len(sort2) < 6:
        pad_req = 6-len(sort2)
        i = 0
        while i < pad_req:
            sort2.append(np.nan) #('n/a')
            i = i+1
        
    #print (top_6)
    #print (sorted(top_6))
    ordered_hist.append(sort2)
    #print ('next item')
    #print ()

In [90]:
medhx_df = pd.DataFrame(ordered_hist, columns = ['medhx' + str(i+1) for i in range(6)])
medhx_df.shape

(102543, 6)

In [91]:
ordered_hist[2000:2020]

[['no significant medical history', nan, nan, nan, nan, nan],
 ['high cholesterol', 'kidney stones', nan, nan, nan, nan],
 ['back pain', nan, nan, nan, nan, nan],
 ['cholesterol', 'benign polyps', nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['high bp', 'hypothyroid', 'chf', nan, nan, nan],
 ['hypertension', 'prostate cancer', 'constipation', nan, nan, nan],
 ['depression', 'asthma', nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['anxiety', 'pneumonia', 'bipolar disorder', nan, nan, nan],
 ['autistic', 'virus in the mouth', 'healed', nan, nan, nan],
 ['no significant medical history',
  'unable to get medical history',
  nan,
  nan,
  nan,
  nan],
 ['hypertension', 'high cholesterol', 'diabetes', nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['no significant medical history', nan, nan, nan, nan, nan],
 ['hypertension',
  'high cholesterol',
  'acid reflux',
  'pacemaker/cardiov

In [92]:
comorbids = []

for item in ordered_hist:
    res = [x for x in item if str(x) != 'nan']
    if len(res) == 0: comorbids.append(np.nan)
    else:
        res = [x for x in res if x != 'no significant medical history']
        #print (res, len(res))
        comorbids.append(len(res))

In [93]:
#so now I want to add this to the tabular data frame
tab_df['num_comorbids'] = comorbids

/Users/jjaskolkambp/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [94]:
clin['num_comorbids'] = comorbids

### some sanity checking

In [18]:
for item in clin['CleanSubjectiveNotes'].iloc[10000:10010]:
    print (item)

patient states feeling depressed for few months,last tuesday patient tried to harm herself by taking 10 extra strength tylenol. denies any intentions to harm herself now or any intention to harm others,
patient was playing around at school, was pushed by fellow student, patient felt a twisting sensation on her left knee and fell to floor. hurts to ambulate on, good csm
as per mother, patient was in the school bus yesterday and it was hit form behind. since then patient has been exp nausea, headaches and stomach aches. no vomits or diarrhea. no urinary or bowel symptoms. no neuro deficits noted. no resp distres.
sst fell on ladder this am,landed to right foot, complains of pain with swelling, sts no head injury, concerned abt his foot, sts took tylenol #3 2 tabs, abt 10 am.sts unable to wieght bear.
as per patient with pain to lateral left back (describes as stabbing pain) radiating across mid back x1 month. seen at wic and bw done and patient sent to emergency department with elevated 

## split the screening questions
- result is a df called: screening_df

In [16]:
qlist = ['Are you feeling feverish or have had shakes or chills in the last 24 hours?',
         'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
        'Do you have a new Rash?',
        'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
         'Have you travelled outside of Canada/USA in the last 3 weeks?',
         'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
         'Have you received Health Care in another country in the last 2 years?',
        'Do you have a new/worse cough or shortness of breath?',
         'If so, select all countries that apply',
        'If so, select all infectious diseases that apply']

In [17]:
#making a dict so that the indect of an item on the list, matches the question later
q_dict = {i:q for i,q in enumerate(qlist)}

In [18]:
q_dict

{0: 'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
 1: 'Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?',
 2: 'Do you have a new Rash?',
 3: 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?',
 4: 'Have you travelled outside of Canada/USA in the last 3 weeks?',
 5: 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?',
 6: 'Have you received Health Care in another country in the last 2 years?',
 7: 'Do you have a new/worse cough or shortness of breath?',
 8: 'If so, select all countries that apply',
 9: 'If so, select all infectious diseases that apply'}

In [19]:
#this is going to be a list made of up all the entries in the infection control screening column
#casting the results as string to manipulate later
qans = list(clin['InfectionControlScreening'].str[14:].astype('str'))

In [20]:
#this is going to give me a way to split the questions by finding where they occur in the field
split_points = []
for item in qans:
    temp_points = []
    for word in qlist:
        temp_points.append(item.find(word))
    split_points.append(temp_points)
    

In [21]:
#this is going to give me list of places to split each entry (using only the questions that are actually present)
real_points = []
for item in split_points:
    real_points.append([idx for idx in sorted(item) if idx > 0])

In [22]:
split_points[10000:10010]

[[241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [290, -1, 449, 374, 0, 119, -1, 228, 70, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1],
 [241, -1, 400, 325, 0, 70, -1, 179, -1, -1]]

In [23]:
#anything from the list above which would have been empty in the original data
#gives a list of -1's in split_points and nothing in real_points
real_points[10000:10010]

[[70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 119, 228, 290, 374, 449],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400],
 [70, 179, 241, 325, 400]]

In [24]:
print(qans[1000])
print()
print (split_points[1000])
print ()
print(real_points[1000])

Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>Do you have a new/worse cough or shortness of breath? Y<LT>LF>Are you feeling feverish or have had shakes or chills in the last 24 hours? Y<LT>LF>Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>Do you have a new Rash? N

[241, -1, 400, 325, 0, 70, -1, 179, -1, -1]

[70, 179, 241, 325, 400]


In [25]:
#this should mutate qans so each item is now a list of q and a's
for idx in range(len(qans)):
    qans[idx] = [qans[idx][i : j] for i, j in zip([0] + real_points[idx], real_points[idx] + [None])]
    

In [26]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Do you have a new/worse cough or shortness of breath? Y<LT>LF>',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? Y<LT>LF>',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>',
 'Do you have a new Rash? N']

In [27]:
qans[10000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N<LT>LF>',
 'Do you have a new/worse cough or shortness of breath? N<LT>LF>',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N<LT>LF>',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N<LT>LF>',
 'Do you have a new Rash? Y']

In [28]:
#this will mutate each item of each list in qans and drop the <LT>LF> if present
for q in qans:
    for idx in range(len(q)):
        if q[idx].find('<LT>LF>') != -1:
            q[idx] = q[idx][:-7]
        

In [29]:
qans[1000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? Y',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? Y',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? N']

In [30]:
qans[10000]

['Have you travelled outside of Canada/USA in the last 3 weeks? N',
 'Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks? N',
 'Do you have a new/worse cough or shortness of breath? N',
 'Are you feeling feverish or have had shakes or chills in the last 24 hours? N',
 'Do you have a new onset of Vomiting/Diarrhea in the last 24 hours? N',
 'Do you have a new Rash? Y']

In [32]:
seg_list = []
for q in qans:
    #print ('initial:', q)
    test_question = q
    res = []
    for item in test_question:
        #print (item)
        for qq in qlist:
            if item.find(qq) != -1:
                #print ('question:',item[:len(qq)])
                #print ('answer:',item[len(qq)+1:])
                res.append([item[:len(qq)],item[len(qq)+1:] ])
    seg_list.append(res)
    #print ('\nresult:',res, '\n\n')

In [33]:
len(seg_list)

102492

In [34]:
seg_list2 = []
for q in qans:
    #print ('initial:', q)
    test_question = q
    res = {}
    for item in test_question:
        #print (item)
        for qq in qlist:
            if item.find(qq) != -1:
                question = item[:len(qq)]
                answer = item[len(qq)+1:]
                res[question] = answer
    seg_list2.append(res)

In [35]:
final_list = []
for q in range(len(split_points)):
    temp = []
    for i in range(len(split_points[q])):
        if split_points[q][i] == -1:
            #print('nan')
            temp.append('nan')
        else:
            #print (brief_list2[q][q_dict[i]])  # q_dict[i], i, brief_splits[q][i],
            temp.append(seg_list2[q][q_dict[i]])
    final_list.append(temp)
    #print ('\n next item \n')
            

In [36]:
len(final_list)

102492

In [37]:
final_list[100:110]

[['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'nan', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan']]

In [38]:
final_list[1000:1020]

[['Y', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'Y', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'Y', 'N', 'nan', 'Y', 'IND', 'nan'],
 ['N', 'nan', 'N', 'nan', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['Y', 'nan', 'N', 'N', 'Y', 'N', 'nan', 'N', 'IND', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N', 'N', 'nan', 'N', 'nan', 'nan'],
 ['N', 'nan', 'N', 'N', 'N

In [39]:
screening_df = pd.DataFrame(final_list, columns = qlist)
screening_df.head()

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
0,Y,nan,N,N,N,N,nan,N,nan,nan
1,Y,nan,N,Y,N,N,nan,N,nan,nan
2,N,nan,N,N,N,N,nan,N,nan,nan
3,N,nan,N,N,N,N,nan,Y,nan,nan
4,N,nan,N,N,N,N,nan,N,nan,nan


In [40]:
screening_df.shape

(102492, 10)

In [41]:
screening_df.iloc[80000:80010]

,Are you feeling feverish or have had shakes or chills in the last 24 hours?,Have you ever been isolated/required isolation for an infectious disease when receiving care in a healthcare setting?,Do you have a new Rash?,Do you have a new onset of Vomiting/Diarrhea in the last 24 hours?,Have you travelled outside of Canada/USA in the last 3 weeks?,Have you had contact with a sick person who has travelled outside of Canada/USA in the last 3 weeks?,Have you received Health Care in another country in the last 2 years?,Do you have a new/worse cough or shortness of breath?,"If so, select all countries that apply","If so, select all infectious diseases that apply"
80000,N,nan,nan,nan,N,nan,nan,N,nan,nan
80001,N,nan,nan,nan,N,nan,nan,N,nan,nan
80002,N,nan,nan,nan,N,nan,nan,N,nan,nan
80003,N,nan,nan,nan,N,nan,nan,N,nan,nan
80004,N,nan,nan,nan,N,nan,nan,N,nan,nan
80005,Y,nan,nan,nan,N,nan,nan,N,nan,nan
80006,nan,nan,nan,nan,N,nan,nan,N,nan,nan
80007,N,nan,nan,nan,N,nan,nan,N,nan,nan
80008,Y,nan,nan,nan,N,nan,nan,Y,nan,nan
80009,Y,nan,nan,nan,N,nan,nan,Y,nan,nan


In [42]:
screening_df.replace('nan',np.NaN, inplace = True)

In [72]:
screening_df.shape

(102492, 10)

In [73]:
clin.shape

(102492, 40)

In [80]:
full = pd.concat([clin,screening_df], axis = 1).reindex(clin.index); full.shape

(102492, 50)

## making the new target cells

In [36]:
admit_count= Counter(clin['AdmitLocation']) #; admit_count

In [37]:
clin['outcome'] = clin['AdmitLocation'].astype('str').map(
{'N.CC CCU': 'madmit',
 'N.CC ICU': 'ICU',
 'N.ER IN' : 'madmit',
 'N.ERMH IN': 'madmit',
 'N.MAU': 'madmit',
 'N.MED CARD': 'madmit',
 'N.MED CPU': 'madmit',
 'N.MED CT': 'madmit',
 'N.MED GER': 'madmit',
 'N.MED DIAL': 'madmit',
 'N.MED NEUR': 'madmit',
 'N.MED ONC': 'madmit',
 'N.MED RESP': 'madmit',
 'N.MED SIM': 'madmit',
 'N.MED SIX': 'madmit',
 'N.MH CHAD': 'madmit',
 'N.MH CHADI': 'madmit',
 'N.MH GENB': 'madmit',
 'N.MH GER': 'madmit',
 'N.MH INTEN': 'madmit',
 'N.SIMCU': 'ICU',
 'N.SUR GEN': 'sadmit',
 'N.SUR ORTH': 'sadmit',
 'N.SUR SS': 'sadmit',
 'N.WC PAED': 'madmit',
 'N.WC POST': 'madmit',
 'N.MH GENC': 'madmit',
 'N.CC NICU': 'ICU',
 'N.WC LD': 'sadmit',
  'N.WC LDN': 'sadmit',
  'N.MAIN OR': 'sadmit',
  'N.MED STTU': 'madmit',
  'N.MED FLEX': 'madmit',
    'N.MED DSU': 'madmit',
    'N.SUR DSU': 'sadmit',
 'nan': 'discharge'})

In [38]:
#first target columns is based on the admitting location
clin['target'] = clin['outcome'].map({'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4})

In [39]:
outcome_count = Counter(clin['outcome']); outcome_count

Counter({'discharge': 92697, 'sadmit': 1490, 'madmit': 8047, 'ICU': 258})

In [40]:
target_count = Counter(clin['target']); target_count

Counter({1: 92697, 3: 1490, 2: 8047, 4: 258})

In [41]:
clin['service'] = clin['PatientService'].astype('str').map(
    {'nan': 'discharge',
         'General Surgery': 'sadmit',
         'Respirology': 'madmit',
         'General Medicine': 'madmit',
         'INTENSIVE CARE UNIT': 'ICU',
         'Cardiology': 'madmit',
         'Oncology': 'madmit',
         'Palliative': 'madmit',
         'AD PALLIATIVE': 'madmit',
         'Genitourinary': 'sadmit',
         'Paediatrics': 'madmit',
         'Mental Health': 'madmit',
         'Orthopaedics': 'sadmit',
         'MH Child & Adolescent': 'madmit',
         'AD Oncology': 'madmit',
         'Neurology': 'madmit',
         'ALC General Medicine': 'madmit',
         'ALC General Surgery': 'sadmit',
         'Acute Care for Elders': 'madmit',
         'ALC Neurology': 'madmit',
         'ALC Cardiology': 'madmit',
         'ALC Respirology': 'madmit',
         'ALC Orthopaedics': 'madmit',
         'Neonatal Retro Transfers': 'madmit',
         'Newborn': 'madmit',
         'Obstetrics': 'madmit',
         'ALC Oncology': 'madmit',
         'AD General Medicine': 'madmit',
         'AD Acute Care for Elders': 'madmit',
         'AD Respirology': 'madmit',
         'ALC ACUTE CARE FOR EDLERS': 'madmit',
         'AD Cardiology': 'madmit',
         'AD Neurology': 'madmit',
         'ALC Palliative': 'madmit',
         'AD General Surgery': 'sadmit'})

In [42]:
service_count = Counter(clin['service'])
service_count

Counter({'discharge': 92697, 'sadmit': 1725, 'madmit': 7900, 'ICU': 170})

In [43]:
#second target is based on the admitting service
clin['target2'] = clin['service'].map({'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4})

In [44]:
target2_count = Counter(clin['target2']); target2_count

Counter({1: 92697, 3: 1725, 2: 7900, 4: 170})

In [45]:
#creating this for later use to map the numbers in the dataframe column to their original meaning
out_dict = {'discharge': 1, 'madmit': 2, 'sadmit': 3, 'ICU': 4}
out_dict = {v:k for k,v in out_dict.items()}
out_dict

{1: 'discharge', 2: 'madmit', 3: 'sadmit', 4: 'ICU'}

In [46]:
#discharge vs. not target
clin['discharge'] = clin['outcome'].map({'discharge': 'discharge', 'madmit': 'admit', 'sadmit': 'admit', 'ICU': 'admit'})

In [47]:
clin['target3'] = clin['outcome'].map({'discharge': 1, 'madmit': 0, 'sadmit': 0, 'ICU': 0})

In [48]:
Counter(clin['discharge']), Counter(clin['target3'])

(Counter({'discharge': 92697, 'admit': 9795}), Counter({1: 92697, 0: 9795}))

In [49]:
#a target for trying to tell apart the inpatient disposition, only in admitted patients
clin['dispo']= clin['outcome'].map({'discharge': np.nan, 'madmit': 'madmit', 'sadmit': 'sadmit', 'ICU': 'ICU'})

In [50]:
clin['target4'] = clin['dispo'].map({np.nan:np.nan, 'madmit': 1, 'sadmit': 2, 'ICU': 3})
#for some reason a can't make a counter out of this without it crashing so I've stopped trying

In [51]:
Counter(clin['dispo']), Counter(clin['target4'].dropna())

(Counter({nan: 92697, 'sadmit': 1490, 'madmit': 8047, 'ICU': 258}),
 Counter({2.0: 1490, 1.0: 8047, 3.0: 258}))

In [52]:
#gonna make a target that is ICU vs other
clin['ICUvsother'] = clin['PatientService'][clin['PatientService'] == 'INTENSIVE CARE UNIT']
#clin['PatientService']

In [53]:
clin['target5'] = clin['ICUvsother'].map({'INTENSIVE CARE UNIT':1, np.nan:0})

### merging to a full dataset

In [54]:
clin.shape

(102492, 41)

In [55]:
clin.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading',
       'CleanSubjectiveNotes', 'CleanMedicalHistory', 'pmhx', 'num_comorbids',
       'outcome', 'target', 'service', 'target2', 'discharge', 'target3',
       'dispo', 'target4', 'ICUvsother', 'target5'],
      dtype='object')

In [56]:
screening_df.shape

NameError: name 'screening_df' is not defined

In [57]:
full = pd.concat([clin,screening_df], axis = 1).reindex(clin.index); full.shape

NameError: name 'screening_df' is not defined

In [134]:
full.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading', 'o2sat', 'pulse',
       'temp', 'CleanSubjectiveNotes', 'BP', 'systolic', 'diastolic', 'Gender',
       'outcome', 'target', 'service', 'pmhx', 'num_comorbids', 'target2',
       'discharge', 'target3', 'dispo', 'target4', 'ICUvsother', 'target5',
       'CleanMedicalHistory',
       'Are you feeling feverish or have had shakes or chills in the last 24 hours?',
      

## splitting into a bunch of sub dataframes
- later will save them as their own csv files
- loading the excel file is very slow, but loading csv files is faster so I'm making a bunch of dataframes of subsets of data for easy saving/loading/combining

In [67]:
tab_df = clin[['ID', 'TriageLevel',
       'AgeInYrs', 'GenderDesc', 'DischargeDisposition',
       'PresentingComplaint', 'PresentingComplaintDesc', 'AdmitLocation', 'PatientService',
       'BloodPressure_LastEDReading','systolic', 'diastolic','temp','pulse','o2sat']]

In [68]:
target_df = clin[['outcome','target', 'service','target2', 'discharge', 'target3','dispo','target4', 'ICUvsother', 'target5']]

In [69]:
date_df = clin[['Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'Left ED Date & Time']]

In [68]:
nlp_df = clin[['CleanSubjectiveNotes', 'pmhx', 'num_comorbids', 'outcome', 'target',
       'service', 'target2', 'discharge', 'target3', 'dispo', 'target4',
       'ICUvsother', 'target5']]

In [69]:
nlp_df.head()

,CleanSubjectiveNotes,pmhx,num_comorbids,outcome,target,service,target2,discharge,target3,dispo,target4,ICUvsother,target5
148,patient states that she feels shaky patient de...,no significant medical history,0,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0
156,,substance misuse,1,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0
197,patient states no pain,no significant medical history,0,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0
217,denied chest pain,no significant medical history,0,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0
218,denies pain eating emergency department lunch ...,"dialysis haemo, type one diabetes, chronic ren...",5,discharge,1,discharge,1,discharge,1,NaN,NaN,NaN,0


## now I'm going to make a couple of balanced datasets

In [135]:
Counter(clin.target), Counter(clin.target3)

(Counter({1: 92697, 3: 1490, 2: 8047, 4: 258}), Counter({1: 92697, 0: 9795}))

In [136]:
admit_df = clin[clin.target3 == 0]

dc_df = clin[clin.target3==1]
dc_df = dc_df.sample(n= len(admit_df))  #this is the number of admissions from target 3



In [137]:
balanced_df1 = pd.concat([admit_df, dc_df], axis = 0)

In [138]:
icu_df = clin[clin.target2 == 4]
madmit_df = clin[clin.target2 ==3].sample(n = len(icu_df))
sadmit_df = clin[clin.target2 ==2].sample(n = len(icu_df))
disc_df = clin[clin.target2 == 1].sample(n = len(icu_df))

In [139]:
#this is a dataframe with equal numbers of all 4 classes
balanced_df2 = pd.concat([icu_df, madmit_df, sadmit_df, disc_df], axis = 0)

In [140]:
non_icu_df = clin[clin.target2 !=4].sample(n = len(icu_df))

In [141]:
balanced_df3 = pd.concat([icu_df, non_icu_df], axis = 0)

In [142]:
balanced_df3.columns

Index(['ID', 'ChartNumber', 'EncounterNumber', 'TriageLevel', 'AgeNumber',
       'AgeInYrs', 'GenderDesc', 'Triage Date & Time', 'Reg Date & Time',
       'PIA Date & Time', 'Disposition Date & Time', 'DischargeDisposition',
       'DischargeDispositionDesc', 'Left ED Date & Time',
       'PresentingComplaint', 'PresentingComplaintDesc', 'MainDiagnosisCode',
       'MainDiagnosisCodeDesc', 'AdmitLocation', 'PatientService',
       'SubjectiveNotes', 'InfectionControlScreening', 'MedicalHistory',
       'BloodPressure_LastEDReading', 'O2Saturation_LastEDReading',
       'Pulse_LastEDReading', 'Temperature_LastEDReading', 'o2sat', 'pulse',
       'temp', 'CleanSubjectiveNotes', 'BP', 'systolic', 'diastolic', 'Gender',
       'outcome', 'target', 'service', 'pmhx', 'num_comorbids', 'target2',
       'discharge', 'target3', 'dispo', 'target4', 'ICUvsother', 'target5',
       'CleanMedicalHistory'],
      dtype='object')

## saving dataframes to csv files

In [70]:

nlp_df.to_csv(data_path + '/clean_nlp_data_bch.csv')

In [70]:
balanced_df1.to_csv(data_path + '/balanced_admit_dc_nlp_data.csv')

balanced_df2.to_csv(data_path +'/balanced_4cls_nlp_data.csv')

balanced_df3.to_csv(data_path +'/balanced_icuvsother_nlp_data.csv')

full.to_csv(data_path +'/complete_clean_bch_data.csv')


#target_df.to_csv(data_path  + '/targets.csv')

#screening_df.to_csv(data_path + '/inf_control_data.csv')

#tab_df.to_csv(data_path + '/tabular_data.csv')

#date_df.to_csv(data_path + '/data_data.csv')

#subj_df.to_csv(data_path + '/subj_data.csv')

#medhx_df.to_csv(data_path + '/med_hx.csv')

NameError: name 'balanced_df1' is not defined